In [1]:
import pandas as pd
import numpy as np

In [2]:
data_dir = "./data/FT_Camp_2/"
inter_dir = "./inter_data_repo/2/"

## 预处理

In [3]:
is227 = True

In [4]:
if is227:
    features_dir = './features_227_repo/'
else:
    features_dir = './features_306_repo/'

In [6]:
# 收支交易数据
sz_detail = pd.read_csv(data_dir + 'sz_detail.csv')
# 类别映射表
trx_cod = pd.read_csv(data_dir + 'trx_cod.csv')
# G2交易代码映射表
g2 = pd.read_csv(data_dir + 'g2.csv')
# 用户基础属性
cust_bas_inf = pd.read_csv(data_dir + 'cust_bas_inf.csv')
# 训练数据
train = pd.read_csv(data_dir + 'train.csv')
# 预测目标用户
pred_users = pd.read_csv(data_dir + 'pred_users.csv')
# 后续处理

In [7]:
tmp1 = set(sz_detail[sz_detail['prt_dt']<='2019-02-27']['g2_cod'].fillna('nan'))
tmp2 = set(sz_detail[sz_detail['prt_dt']>='2019-01-08']['g2_cod'].fillna('nan'))
g2_cods = list(tmp1 & tmp2)
g2_cods.sort()

In [8]:
print(len(tmp1))
print(len(tmp2))
print(len(g2_cods))

360
362
359


In [9]:
if is227:
    sz_detail = sz_detail[sz_detail['prt_dt']<='2019-02-27']
else:
    sz_detail = sz_detail[sz_detail['prt_dt']>='2019-01-08']

In [10]:
# dates = list(set(sz_detail['prt_dt']))
# dates.sort()
# print(len(dates))

In [11]:
sz_detail.shape

(8161655, 5)

In [12]:
print(sz_detail.loc[:,'prt_dt'].min())
print(sz_detail.loc[:,'prt_dt'].max())

2019-01-01
2019-02-27


In [13]:
# sz_detail.loc[:,'prt_dt']

In [14]:
# sz_detail[sz_detail['prt_dt']=='2019-01-01']

## 特征提取

In [15]:
# 收支记录次数
tmp_df = sz_detail.groupby(['id']).size()
tmp_df = tmp_df.to_frame()
cname = "收支记录次数"
tmp_df.columns = [cname]
tmp_df.to_csv(features_dir + cname+'.csv')
tmp_df.head()

,收支记录次数
id,
1,13
2,31
3,167
4,50
5,41


In [16]:
# 收支总和
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df.groupby(['id']).sum()
tmp_df.columns = ['收支总和']
tmp_df.to_csv(features_dir + '收支总和.csv')

In [17]:
# 收入总和
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df[tmp_df['rmb_amt']>=0]
tmp_df = tmp_df.groupby(['id']).sum()
tmp_df.columns = ['收入总和']
tmp_df.to_csv(features_dir + '收入总和.csv')

In [18]:
# 支出总和
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df[tmp_df['rmb_amt']<0]
tmp_df = tmp_df.groupby(['id']).sum()
tmp_df.columns = ['支出总和']
tmp_df.to_csv(features_dir + '支出总和.csv')

In [19]:
# 收入最大值
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df[tmp_df['rmb_amt']>=0]
tmp_df = tmp_df.groupby(['id']).max()
tmp_df.columns = ['收入最大值']
tmp_df.to_csv(features_dir + '收入最大值.csv')
tmp_df.head()

,收入最大值
id,
1,56388.00
2,96000.00
3,6841.06
4,520.00
5,26941.90


In [20]:
# 支出最大值
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df[tmp_df['rmb_amt']<0]
tmp_df = tmp_df.groupby(['id']).min()
tmp_df.columns = ['支出最大值']
tmp_df.to_csv(features_dir + '支出最大值.csv')
tmp_df.head()

,支出最大值
id,
1,-50000.0
2,-316000.0
3,-3000.0
4,-520.0
5,-26941.0


In [21]:
# 用户基础属性
tmp_df = cust_bas_inf.copy()
def _convert_gender(x):
    if x=='M':
        return 0
    elif x=='F':
        return 1
    else:
        return -1
tmp_df['gender'] = tmp_df['gender'].apply(_convert_gender)    
# tmp_df.replace('\\N','0', inplace = True)    # TODO: 年龄用平均数代替
tmp_df.loc[tmp_df['age']=='\\N', 'age'] = np.nan
tmp_df.loc[tmp_df['aum227']=='\\N', 'aum227'] = np.nan
# tmp_df.loc[tmp_df['aum227']=='-9999', 'aum227'] = '0'
tmp_df.loc[tmp_df['aum306']=='\\N', 'aum306'] = np.nan
# tmp_df.loc[tmp_df['aum306']=='-9999', 'aum306'] = '0'
tmp_df['age'] = tmp_df['age'].astype("float")
tmp_df['aum227'] = tmp_df['aum227'].astype("float")
tmp_df['aum306'] = tmp_df['aum306'].astype("float")

if is227:
    tmp_df = tmp_df[["id","gender","age","aum227"]]
    tmp_df.columns = ["id","gender","age","aum227"]
    tmp_df.to_csv(features_dir + "用户基础属性.csv", columns = ["id","gender","age","aum227"], index=False)
else:
    tmp_df = tmp_df[["id","gender","age","aum306"]]
    tmp_df.columns = ["id","gender","age","aum227"]
    tmp_df.to_csv(features_dir + "用户基础属性.csv", columns = ["id","gender","age","aum227"], index=False)
tmp_df.head()

,id,gender,age,aum227
0,1,0,52.0,219481.95
1,2,0,32.0,236183.79
2,3,0,22.0,0.06
3,4,1,21.0,420.26
4,5,0,32.0,149.65


In [22]:
# %%time
# 各类别收支记录次数
sz_ids = list(trx_cod['sz_id'])
user_ids = list(set(sz_detail['id']))

tmp_rst_dict = dict()
base_d = dict(zip(sz_ids, [0]*len(sz_ids)))
for uid in user_ids:
    tmp_rst_dict[uid] = base_d.copy()

tmp_df = sz_detail.groupby(['id','sz_id']).size()

tmp_df = tmp_df.to_frame()

for m,n in tmp_df.iterrows():
    x = m[0]
    y = m[1]
#     print(x,y)
    tmp_rst_dict[x][y] = n.to_list()[0]
#     print(tmp_rst_dict[x][y])
tmp_rst = pd.DataFrame.from_dict(tmp_rst_dict, orient='index')
tmp_rst['id'] = tmp_rst.index
tmp_rst.to_csv(features_dir + '各类别收支记录次数.csv', index=False)
tmp_rst.head()

,101,102,103,104,105,106,107,108,109,110,...,303,304,305,306,307,308,309,310,311,id
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,2
3,0,6,4,1,0,0,0,3,0,0,...,0,0,0,0,0,4,0,1,0,3
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,4
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,4,7,0,0,5


In [23]:
# %%time
# 各类别收支金额
sz_ids = list(trx_cod['sz_id'])
user_ids = list(set(sz_detail['id']))

tmp_rst_dict = dict()
base_d = dict(zip(sz_ids, [0]*len(sz_ids)))
for uid in user_ids:
    tmp_rst_dict[uid] = base_d.copy()
    
tmp_df = sz_detail[['id','sz_id','rmb_amt']]
tmp_df = tmp_df.groupby(['id','sz_id']).sum()
# tmp_df = tmp_df.to_frame()
for m,n in tmp_df.iterrows():
    x = m[0]
    y = m[1]
#     print(x,y)
    tmp_rst_dict[x][y] = n.to_list()[0]
tmp_rst = pd.DataFrame.from_dict(tmp_rst_dict, orient='index')
tmp_rst['id'] = tmp_rst.index
tmp_rst.to_csv(features_dir + '各类别收支金额.csv', index=False)
tmp_rst.head()

,101,102,103,104,105,106,107,108,109,110,...,303,304,305,306,307,308,309,310,311,id
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,-50000.00,0.00,0.0,0.0,1
2,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,34845.86,0.00,0.0,0.0,2
3,0.0,-120.5,-204.0,-3.0,0.0,0.0,0.0,-179.64,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,-55.25,0.00,-1.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-49.90,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,-35.00,0.00,0.0,0.0,4
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,-27111.72,-8297.00,-35206.44,0.0,0.0,5


In [24]:
# %%time
# 各原始类别收支记录次数
# g2_cods = list(set(sz_detail['g2_cod'].fillna('nan')))
# g2_cods.sort()
user_ids = list(set(sz_detail['id']))
user_ids.sort()

tmp_rst_dict = dict()
base_d = dict(zip(g2_cods, [0]*len(g2_cods)))
for uid in user_ids:
    tmp_rst_dict[uid] = base_d.copy()

tmp_df = sz_detail.groupby(['id','g2_cod']).size()

tmp_df = tmp_df.to_frame()

for m,n in tmp_df.iterrows():
    x = m[0]
    y = m[1]
    tmp_rst_dict[x][y] = n.to_list()[0]
    
tmp_rst = pd.DataFrame.from_dict(tmp_rst_dict, orient='index')
tmp_rst['id'] = tmp_rst.index
dc = set(tmp_rst.columns)-{'id'} - set(g2_cods)
tmp_rst = tmp_rst.drop(dc, axis=1)
tmp_rst.to_csv(features_dir + '各原始类别收支记录次数.csv', index=False)
print(tmp_rst.head())

   1029  1050  3200  8001  ABFW  ACTF  ACZZ  AGAF  AGAM  AGAN  ...  YDFS  \
1     0     0     0     0     0     0     0     0     0     0  ...     0   
2     0     0     0     0     0     0     0     0     0     0  ...     0   
3     0     0     0     0     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   
5     0     0     0     0     0     0     0     0     0     0  ...     0   

   YDGS  YEPC  YEPD  YFTB  YKHK  YKJJ  YKZZ  nan  id  
1     0     0     0     0     0     0     0    0   1  
2     0     0     0     0     0     0     0    0   2  
3     0     0     0     0     0     0     0    0   3  
4     0     0     0     0     0     0     0    0   4  
5     0     0     0     0     0     0     0    0   5  

[5 rows x 360 columns]


In [25]:
# %%time
# 各原始类别收支金额
# g2_cods = list(set(sz_detail['g2_cod'].fillna('nan')))
# g2_cods.sort()
user_ids = list(set(sz_detail['id']))
user_ids.sort()

tmp_rst_dict = dict()
base_d = dict(zip(g2_cods, [0]*len(g2_cods)))
for uid in user_ids:
    tmp_rst_dict[uid] = base_d.copy()

tmp_df = sz_detail[['id','g2_cod','rmb_amt']]
tmp_df = tmp_df.groupby(['id','g2_cod']).sum()

for m,n in tmp_df.iterrows():
    x = m[0]
    y = m[1]
#     print(x,y)
    tmp_rst_dict[x][y] = n.to_list()[0]
tmp_rst = pd.DataFrame.from_dict(tmp_rst_dict, orient='index')
tmp_rst['id'] = tmp_rst.index
dc = set(tmp_rst.columns)-{'id'} - set(g2_cods)
tmp_rst = tmp_rst.drop(dc, axis=1)
tmp_rst.to_csv(features_dir + '各原始类别收支金额.csv', index=False)
print(tmp_rst.head())

   1029  1050  3200  8001  ABFW  ACTF  ACZZ  AGAF  AGAM  AGAN  ...  YDFS  \
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   YDGS  YEPC  YEPD  YFTB  YKHK  YKJJ  YKZZ  nan  id  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0   1  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0   2  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0   3  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0   4  
5   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0   5  

[5 rows x 360 columns]


In [ ]:
# 收支均值
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df.groupby(['id']).mean()
tmp_df.columns = ['收支均值']
tmp_df.to_csv(features_dir + '收支均值.csv')
tmp_df.head()

,收支均值
id,
1,3714.307692
2,272.719677
3,-33.775689
4,0.321400
5,1.307317


In [ ]:
# 收入均值
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df[tmp_df['rmb_amt']>=0]
tmp_df = tmp_df.groupby(['id']).mean()
tmp_df.columns = ['收入均值']
tmp_df.to_csv(features_dir + '收入均值.csv')
tmp_df.head()

,收入均值
id,
1,20145.500000
2,20511.865294
3,2306.890000
4,124.117647
5,9460.630000


In [ ]:
# 支出均值
tmp_df = sz_detail.loc[:,['id','rmb_amt']]
tmp_df = tmp_df[tmp_df['rmb_amt']<0]
tmp_df = tmp_df.groupby(['id']).mean()
tmp_df.columns = ['支出均值']
tmp_df.to_csv(features_dir + '支出均值.csv')
tmp_df.head()

,支出均值
id,
1,-10369.571429
2,-24303.385714
3,-76.592744
4,-63.452424
5,-6052.659200
